# Evidently for Drift Monitoring

In [1]:
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import TextEvals
from evidently.descriptors import *

In [4]:
from evidently.ui.workspace.cloud import CloudWorkspace
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from.env file

ws = CloudWorkspace(token=os.environ.get("EVIDENTLY_API_KEY"), url="https://app.evidently.cloud")

In [6]:
project = ws.create_project("FAQ RAG for Law Docs", team_id="019315b7-2eda-77d5-9614-ffd7624194b9")
project.description = "This is a trial project for exploring FAQ RAG and integrating evidently for model drift evaluation."
project.save()

Project(id=UUID('019315b8-8594-7683-b69b-e2e0d897358a'), name='FAQ RAG for Law Docs', description='This is a trial project for exploring FAQ RAG and integrating evidently for model drift evaluation.', dashboard=DashboardConfig(name='FAQ RAG for Law Docs', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=UUID('019315b7-2eda-77d5-9614-ffd7624194b9'), date_from=None, date_to=None, created_at=datetime.datetime(2024, 11, 10, 10, 57, 14, 645204))

In [8]:
from base import BASE_DIR

df = pd.read_parquet(f"{BASE_DIR}/output/main/test-001/data.parquet")
df.shape

(15, 12)

In [12]:
columns = ["question", "answer"]
evaluation_dataset = df[columns]

text_evals_report = Report(metrics=[
    TextEvals(column_name="answer", descriptors=[
        Sentiment(),
        TextLength(),
        IncludesWords(words_list=['sorry', 'apologize'], display_name="Denials"),
        ]
    ),
])

text_evals_report.run(reference_data=None, current_data=evaluation_dataset)

In [13]:
ws.add_report(project.id, text_evals_report, include_data=True)